<center><font size="+4">Programming & Data Analytics & AI 2 PM 2024/2025</font></center>
<center><font size="+2">Sant'Anna School of Advanced Studies, Pisa, Italy</font></center>
<center><img src="https://github.com/EMbeDS-education/ComputingDataAnalysisModeling20242025/raw/main/PDAI/jupyter/jupyterNotebooks/images/sssaLEMBEDSdtu.png" width="900" alt="L'EMbeDS"></center>

<center><font size="+2">Course responsible</font></center>
<center><font size="+2">Andrea Vandin a.vandin@santannapisa.it</font></center>

---

<center><font size="+2">Notebook for Class 6 </font></center>
<center><font size="+2">Petri nets and Alpha miner. Data Lab
</font></center>

---
---

In [2]:
url_github_repo="https://github.com/EMbeDS-education/ComputingDataAnalysisModeling20242025/raw/main/PDAI/"

How good are you with pm4py?
- Let's test it! :D

In [3]:
from IPython.display import IFrame
IFrame("https://kahoot.it/", 500, 400)

For an informed discussion on this topic, I recommend you watching classes from 2.2 to 2.7 of the [Coursera course on PM](https://www.coursera.org/learn/process-mining/home/welcome)
- https://www.coursera.org/learn/process-mining/lecture/XGSCe/2-2-petri-nets-1-2



- (2.2, 2.3) Examples on Petri nets 
- (2.4)
  - Boundedness and safeness 
  - Deadlocks in Petri net
  - Liveness
  - Transition system, Traces, Complete traces
- (2.5)
  - Workflow nets (1 start state, 1 final state)
  -  Soundness
    - Safe: places cannot hold multiple tokens at the same time
    - Proper completion: if the sink place is marked, all other places are empty
      - Implied by Option to complete
    - Option to complete: it is always possible to reach the marking that marks just the sink place
    - Deadlock-free
  - A WF-net is sound iff the corresponding short-circuited PN is live and bounded
- (2.6): We learn WF-nets with Alpha miner

> Below we provide screenshots from some slides used above

In [3]:
pm_slides=url_github_repo+'jupyter/jupyterNotebooks/images/PM/'

## (2.6): We learn WF-nets with Alpha miner

In [ ]:
img=Image(pm_slides+'lastKahootMeme.JPG',width=700)